In [1]:
import os
import numpy as np
from PIL import Image

In [2]:
def read_files(folder_path, mask_path):

    tiff_files = [file for file in os.listdir(folder_path) if file.endswith('.tiff')]

    images = []
    masks = []
    for file in tiff_files:
        image = Image.open(os.path.join(folder_path, file))
        images.append(np.array(image))
        mask = Image.open(os.path.join(mask_path, os.path.splitext(file)[0] + '_mask.png'))
        masks.append(np.array(mask))

    return np.array(images), np.array(masks)

In [3]:
def divide_into_patches(images, patch_size, stride):
    num_images, height, width = images.shape
    
    patches_per_dim_h = (height - patch_size) // stride + 1
    patches_per_dim_w = (width - patch_size) // stride + 1
    
    num_patches_per_image = patches_per_dim_h * patches_per_dim_w
    
    patches = np.zeros((num_images * num_patches_per_image, patch_size, patch_size), dtype=images.dtype)
    
    idx = 0
    for image in images:
        for h in range(0, height - patch_size + 1, stride):
            for w in range(0, width - patch_size + 1, stride):
                patch = image[h:h+patch_size, w:w+patch_size]
                patches[idx] = patch
                idx += 1
                
    return patches

In [4]:
def data_split(data, train_ratio=0.7, test_ratio=0.2, val_ratio=0.1):
    num_samples = data.shape[0]
    
    train_size = int(train_ratio * num_samples)
    val_size = int(val_ratio * num_samples)
    
    train_indices = np.arange(0, train_size)
    val_indices = np.arange(train_size, train_size+val_size)
    test_indices = np.arange(train_size+val_size, num_samples)
    
    train_set = data[train_indices]
    test_set = data[test_indices]
    val_set = data[val_indices]
    
    return train_set, test_set, val_set

In [5]:
def penobscot_data(patch,stride):
    inlines, masks_in=read_files("/home/grad/ccomp/21/nuneslima/Datasets/Penobscot/inlines",'/home/grad/ccomp/21/nuneslima/Datasets/Penobscot/masks')
    crosslines, masks_cross=read_files("/home/grad/ccomp/21/nuneslima/Datasets/Penobscot/crosslines",'/home/grad/ccomp/21/nuneslima/Datasets/Penobscot/masks')
    patch_inline=divide_into_patches(inlines,patch,stride)
    patch_crossline=divide_into_patches(crosslines,patch,stride)
    patch_mask_in=divide_into_patches(masks_in,patch,stride)
    patch_mask_cross=divide_into_patches(masks_cross,patch,stride)

    in_train,in_test,in_val=data_split(patch_inline)
    mask_in_train,mask_in_test,mask_in_val=data_split(patch_mask_in)
    cross_train,cross_test,cross_val=data_split(patch_crossline)
    mask_cross_train,mask_cross_test,mask_cross_val=data_split(patch_mask_cross)

    trainX=np.append(in_train,cross_train, axis=0)
    trainY=np.append(mask_in_train,mask_cross_train, axis=0)
    testX=np.append(in_test,cross_test, axis=0)
    testY=np.append(mask_in_test,mask_cross_test, axis=0)
    valX=np.append(in_val,cross_val, axis=0)
    valY=np.append(mask_in_val,mask_cross_val, axis=0)

    return trainX, trainY, testX, testY, valX, valY


In [6]:
inlines, masks_in=read_files("/home/grad/ccomp/21/nuneslima/Datasets/Penobscot/inlines",'/home/grad/ccomp/21/nuneslima/Datasets/Penobscot/masks')
inlines = ((inlines + 32767) / 65534) * 255
inlines=inlines.astype(np.uint8)
#crosslines, masks_cross=read_files("/home/grad/ccomp/21/nuneslima/Datasets/Penobscot/crosslines",'/home/grad/ccomp/21/nuneslima/Datasets/Penobscot/masks')

In [7]:
print(np.unique(inlines))


[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24

In [8]:
print(inlines.shape,crosslines.shape)
print(masks_in.shape,masks_cross.shape)

NameError: name 'crosslines' is not defined

In [ ]:
patch_inline=divide_into_patches(inlines,50,30)
patch_crossline=divide_into_patches(crosslines,50,30)
patch_mask_in=divide_into_patches(masks_in,50,30)
patch_mask_cross=divide_into_patches(masks_cross,50,30)

In [ ]:
print(patch_inline.shape,patch_crossline.shape)
print(patch_mask_in.shape,patch_mask_cross.shape)

(438060, 50, 50) (447811, 50, 50)
(438060, 50, 50) (447811, 50, 50)


In [ ]:
in_train,in_test,in_val=data_split(patch_inline)
print(in_train.shape, in_test.shape, in_val.shape)

mask_in_train,mask_in_test,mask_in_val=data_split(patch_mask_in)
print(mask_in_train.shape, mask_in_test.shape, mask_in_val.shape)

(306642, 50, 50) (87612, 50, 50) (43806, 50, 50)
(306642, 50, 50) (87612, 50, 50) (43806, 50, 50)


In [ ]:
cross_train,cross_test,cross_val=data_split(patch_crossline)
print(cross_train.shape, cross_test.shape, cross_val.shape)

mask_cross_train,mask_cross_test,mask_cross_val=data_split(patch_mask_cross)
print(mask_cross_train.shape, mask_cross_test.shape, mask_cross_val.shape)

(313467, 50, 50) (89563, 50, 50) (44781, 50, 50)
(313467, 50, 50) (89563, 50, 50) (44781, 50, 50)


In [ ]:
trainX=np.append(in_train,cross_train, axis=0)
trainY=np.append(mask_in_train,mask_cross_train, axis=0)
testX=np.append(in_test,cross_test, axis=0)
testY=np.append(mask_in_test,mask_cross_test, axis=0)
valX=np.append(in_val,cross_val, axis=0)
valY=np.append(mask_in_val,mask_cross_val, axis=0)

In [ ]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)
print(valX.shape)
print(valY.shape)

(620109, 50, 50)
(620109, 50, 50)
(177175, 50, 50)
(177175, 50, 50)
(88587, 50, 50)
(88587, 50, 50)


In [9]:
import os
import numpy as np
from PIL import Image

def read_files(folder_path, mask_path):

    tiff_files = [file for file in os.listdir(folder_path) if file.endswith('.tiff')]

    images = []
    masks = []
    for file in tiff_files:
        image = Image.open(os.path.join(folder_path, file))
        images.append(np.array(image))
        mask = Image.open(os.path.join(mask_path, os.path.splitext(file)[0] + '_mask.png'))
        masks.append(np.array(mask))

    return np.array(images), np.array(masks)


def divide_into_patches(images, patch_size, stride):
    num_images, height, width = images.shape
    
    patches_per_dim_h = (height - patch_size) // stride + 1
    patches_per_dim_w = (width - patch_size) // stride + 1
    
    num_patches_per_image = patches_per_dim_h * patches_per_dim_w
    
    patches = np.zeros((num_images * num_patches_per_image, patch_size, patch_size), dtype=images.dtype)
    
    idx = 0
    for image in images:
        for h in range(0, height - patch_size + 1, stride):
            for w in range(0, width - patch_size + 1, stride):
                patch = image[h:h+patch_size, w:w+patch_size]
                patches[idx] = patch
                idx += 1
                
    return patches


def data_split(data, train_ratio=0.7, test_ratio=0.2, val_ratio=0.1):
    num_samples = data.shape[0]
    
    train_size = int(train_ratio * num_samples)
    val_size = int(val_ratio * num_samples)
    
    train_indices = np.arange(0, train_size)
    val_indices = np.arange(train_size, train_size+val_size)
    test_indices = np.arange(train_size+val_size, num_samples)
    
    train_set = data[train_indices]
    test_set = data[test_indices]
    val_set = data[val_indices]
    
    return train_set, test_set, val_set


def majority_class(images, masks, threshold_percentage=0.7):
    n, a, b = masks.shape
    majority_classes = []
    majority_images=[]

    for i in range(n):
        sample = masks[i]
        flattened_sample = sample.flatten()
        unique_classes, counts = np.unique(flattened_sample, return_counts=True)
        max_count = np.max(counts)
        total_count = np.sum(counts)
        if max_count / total_count >= threshold_percentage:
            majority_class_index = np.argmax(counts)
            majority_class = unique_classes[majority_class_index]
            majority_classes.append(majority_class)
            majority_images.append(images[i])


    return np.array(majority_images), np.array(majority_classes)




def penobscot_data_seg(patch,stride):
    inlines, masks_in=read_files("/home/grad/ccomp/21/nuneslima/Datasets/Penobscot/inlines",'/home/grad/ccomp/21/nuneslima/Datasets/Penobscot/masks')
    inlines = ((inlines + 32767) / 65534) * 255
    inlines=inlines.astype(np.uint8)
    #crosslines, masks_cross=read_files("/home/grad/ccomp/21/nuneslima/Datasets/Penobscot/crosslines",'/home/grad/ccomp/21/nuneslima/Datasets/Penobscot/masks')
    patch_inline=divide_into_patches(inlines,patch,stride)
    #patch_crossline=divide_into_patches(crosslines,patch,stride)
    patch_mask_in=divide_into_patches(masks_in,patch,stride)
    #patch_mask_cross=divide_into_patches(masks_cross,patch,stride)

    in_train,in_test,in_val=data_split(patch_inline)
    mask_in_train,mask_in_test,mask_in_val=data_split(patch_mask_in)

    return in_train, mask_in_train, in_test, mask_in_test, in_val, mask_in_val


def penobscot_data(patch,stride):
    inlines, masks_in=read_files("/home/grad/ccomp/21/nuneslima/Datasets/Penobscot/inlines",'/home/grad/ccomp/21/nuneslima/Datasets/Penobscot/masks')
    inlines = ((inlines + 32767) / 65534) * 255
    inlines=inlines.astype(np.uint8)
    #crosslines, masks_cross=read_files("/home/grad/ccomp/21/nuneslima/Datasets/Penobscot/crosslines",'/home/grad/ccomp/21/nuneslima/Datasets/Penobscot/masks')
    patch_inline=divide_into_patches(inlines,patch,stride)
    #patch_crossline=divide_into_patches(crosslines,patch,stride)
    patch_mask_in=divide_into_patches(masks_in,patch,stride)
    #patch_mask_cross=divide_into_patches(masks_cross,patch,stride)

    in_train,in_test,in_val=data_split(patch_inline)
    mask_in_train,mask_in_test,mask_in_val=data_split(patch_mask_in)
    # cross_train,cross_test,cross_val=data_split(patch_crossline)
    # mask_cross_train,mask_cross_test,mask_cross_val=data_split(patch_mask_cross)

    # trainX=np.append(in_train,cross_train, axis=0)
    # trainY=np.append(mask_in_train,mask_cross_train, axis=0)
    # testX=np.append(in_test,cross_test, axis=0)
    # testY=np.append(mask_in_test,mask_cross_test, axis=0)
    # valX=np.append(in_val,cross_val, axis=0)
    # valY=np.append(mask_in_val,mask_cross_val, axis=0)




    patch_train,train_labels = majority_class(in_train, mask_in_train, 0.7)
    patch_test,test_labels = majority_class(in_test, mask_in_test, 0.7)
    patch_val,val_labels = majority_class(in_val, mask_in_val, 0.7)


    return patch_train, train_labels, patch_test, test_labels, patch_val, val_labels

    #return trainX, majority_class(trainY), testX, majority_class(testY), valX, majority_class(valY)


In [ ]:
trainX,trainY,testX,testY,valX,valY=penobscot_data(patch,stride)

In [ ]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)
print(valX.shape)
print(valY.shape)